Treat:
- dm_clientes
- ft_pesquisa_satisfacao

In [0]:
from pyspark.sql import functions as F

In [0]:
catalogo = 'atendimento_catalog'
bronze_schema = 'bronze'
silver_schema = ''



dm_clientes

In [0]:
c = spark.read.table(f'{catalogo}.{bronze_schema}.dm_clientes')

print("ANALISE DA TABELA\n")
count = c.count()
ids_dist = c.select(F.col("id_cliente")).distinct()
count_dist = ids_dist.count()
print(f'Total ids: {count} | Distinct ids: {count_dist}')

regioes = [row['regiao'] for row in c.select("regiao").distinct().collect()]
print(f'regioes: {regioes}')

idade = c.agg(
    F.max("idade").alias("max_idade"),
    F.min("idade").alias("min_idade")
).collect()[0]

print(f"max age: {idade['max_idade']} | min age: {idade['min_idade']}")

s_c = (c
       .select(
           F.col('id_cliente').cast('bigint'),
           F.col('nome').cast('string'),
           F.col('email').cast('string'),
           F.col('regiao').cast('string'),
           F.col('idade').cast('int')   
       )
       .where(F.col('id_cliente').isNotNull())
       .where(F.col('email').isNotNull())
       .where(F.col('email').contains('@'))
       .withColumn('data_ingestao', F.current_timestamp())
    )

nome_outlier = (c 
       .filter((F.length(F.col("nome")) > 20) | (F.size(F.split(F.col("nome"), " ")) > 2))
    )
print('\n\nOutliers:')
print(f'Nome > 20 caracteres: {nome_outlier.count()}')
nome_outlier.display()

print(f'\n\nregistros finais: {s_c.count()}')
print(f'valores nulos nas colunas: ')
print(f'email: {c.filter(F.col("email").isNull()).count()}')
print(f'regiao: {c.filter(F.col("regiao").isNull()).count()}')
print(f'idade: {c.filter(F.col("idade").isNull()).count()}')
s_c.display()

In [0]:
ps = spark.read.table(f'{catalogo}.{bronze_schema}.ft_pesquisa_satisfacao')

print("ANALISE DA TABELA\n")
count = ps.count()
ids_dist = ps.select(F.col("id_pesquisa")).distinct()
count_dist = ids_dist.count()
print(f'Total ids: {count} | Distinct ids: {count_dist}')
print(f'avaliacoes nulas: {ps.filter(F.col("nota_atendimento").isNull()).count()}')

s_ps = (
    ps
    .select(
        F.col('id_pesquisa').cast('bigint'),
        F.col('id_chamado').cast('bigint'),
        F.col('nota_atendimento').cast('int')
    )
    .where(F.col('id_pesquisa').isNotNull())
    .where(F.col('id_chamado').isNotNull())
    .where((F.col('nota_atendimento') <= 5) & (F.col('nota_atendimento') >= 1)) # intervalo valido [1,5]
    .withColumn('data_ingestao', F.current_timestamp())
)
print(f'\n\nregistros finais: {s_ps.count()}')
s_ps.display()